# 08wk-2: Pandas – `groupby`, 데이터프레임의 결합

최규빈  
2023-10-23

<a href="https://colab.research.google.com/github/guebin/DV2023/blob/main/posts/08wk-2.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

[????](????)

# 2. Imports

In [2]:
import pandas as pd 
import numpy as np

# 3. `groupby`

## A. DataFrameGroupBy

`-` `groupby` + `agg` 의 복습

In [18]:
df = pd.DataFrame({'department':['A','A','B','B'], 'gender':['male','female','male','female'],'count':[1,2,3,1]})
df

In [20]:
df.groupby('department').agg({'count':'sum'})

`-` 의문: 위의 코드에서 `df.groupby('department')`는 어떤 object일까?

In [21]:
g = df.groupby('department')
g

In [61]:
set(dir(g)) & {'__iter__'} # iterable object -> 대놓고 반복문쓰리는 의미

In [62]:
list(g) # 반복인자는 아래와 같음 

오브젝트 `g`는

1.  sub-dataframe 이 모여있는 묶음체이이다.
2.  반복문을 돌리기에 용이하도록 고안되어있다. (이때 각 반복인자는
    `(묵음기준,sub-dataframe)`의 튜플로 이루어져 있음)

`-` 제가 이해하는 방식

In [65]:
dct = {k: df for (k,df) in g}
dct

In [66]:
display(dct['A'])
display(dct['B'])

## B. `g`를 이용하는 방법

`# 예시1`: `g`를 이용하여 원래의 df를 복원하라.

In [69]:
df = pd.DataFrame({'department':['A','A','B','B'], 'gender':['male','female','male','female'],'count':[1,2,3,1]})
g = df.groupby('department')

(풀이)

In [70]:
pd.concat([df for (k,df) in g])

`#`

`# 예시2`: `g`를 이용하여 아래와 동일한 기능을 하는 코드를 작성하라.
(agg 함수 사용금지)

In [71]:
df = pd.DataFrame({'department':['A','A','B','B'], 'gender':['male','female','male','female'],'count':[1,2,3,1]})
df.groupby('department').agg({'count':'sum'})

(풀이)

In [72]:
g = df.groupby('department')

In [81]:
pd.DataFrame(pd.Series({k:df['count'].sum() for k,df in g}))

`#`

`# 예시3`: 아래는 데이터프레임이 있다고 치자. 이 데이터프레임을 `class`
를 기준으로 그룹핑하여 sub-dataframe을 만들고 score가 높은 순서로
정렬하는 코드를 작성하라.

In [87]:
df = pd.DataFrame({'class':['A']*5+['B']*5, 'id':[0,1,2,3,4]*2, 'score':[60,20,40,60,90,20,30,90,95,95]})
df

(풀이)

In [88]:
g = df.groupby('class')

In [92]:
pd.concat([df.sort_values('score',ascending = False) for _,df in g])

`#`

# 4. `concat`

In [95]:
pass 

# 5. `merge`

## A. 가장 빈번하게 사용하는 상황

`# 예시1` – `big` dataframe에 `groupby`+`agg`를 사용하여 `small`
dataframe이 생긴경우

In [99]:
big = pd.DataFrame({'department':['A','A','B','B'], 'gender':['male','female','male','female'],'count':[1,2,3,1]})
small = pd.DataFrame({'department':['A','B'], 'total':[3,4]})

In [100]:
display("big",big)
display("small",small)

`-` 정보를 종합하기 위해서는 아래와 같이 하면 된다.

In [106]:
pd.merge(big,small) # 개념을 위해서 추천하는 코드 
big.merge(small) # 실제 사용하는 코드
#---#
# pd.merge(small,big) # 보기싫게 나옴
# small.merge(big) # 보기싫게 나옴

**그런데 사실 아래는 같은 코드이다.**

In [107]:
big.merge(small)
big.merge(small,on='department')

> 상황: 두 데이터프레임은 특정한 **IndexLabel**에 대하여 **서로 다른
> 정보**를 각각 정리한 상황

> 원리: (1) 연결고리파악: 두 데이터프레임에서 공통인 열(IndexLabel)을
> 찾는다. (2) 병합: 연결고리를 기준으로 데이터의 정보를 병합한다.

**이 예시의 경우**

-   IndexLabel = 연결고리 = 공통의열 = `department`
-   서로다른정보들: 이 예제에서 big은 `department`에 대한 gender와
    count를 정리하고 있음. 그리고 small은 `department`에 대한 `total`을
    정리하고 있음. 이는 서로 다른 정보임.

`#`

## B. 여러가지 파라메터

`# 예시1` – on

In [109]:
big = pd.DataFrame({'department':['A','A','B','B'], 'gender':['male','female','male','female'],'count':[1,2,3,1]})
small = pd.DataFrame({'department':['A','B'], 'count':[3,4]})
display("big",big)
display("small",small)

`-` 잘못된 코드

In [111]:
pd.merge(big,small)

`-` 올바른 코드

In [112]:
pd.merge(big,small,on='department')

**그런데 사실 아래는 같은 코드이다.**

In [113]:
pd.merge(big,small,on='department')
pd.merge(big,small,left_on='department',right_on='department')

`*` 더(?) 올바른 코드 – 좀 더 제 스타일의 코드

In [ ]:
big.merge(small.rename({'count':'sum'},axis=1))

`#`

`# 예시2` – left_on, right_on

In [121]:
big = pd.DataFrame({'department':['A','A','B','B'], 'gender':['male','female','male','female'],'count':[1,2,3,1]})
small = pd.DataFrame({'dept':['A','B'], 'count':[3,4]})
display("big",big)
display("small",small)

-   공통의 열이 오히려 ‘count’

In [122]:
pd.merge(big,small)

In [123]:
pd.merge(big,small,left_on='department',right_on='dept')

In [124]:
pd.merge(small,big,left_on='dept',right_on='department')

`*` 좀 더 제 스타일의 코드

In [129]:
display(small.set_axis(['department','sum'],axis=1))
display(big)

In [130]:
pd.merge(big,small.set_axis(['department','sum'],axis=1))

`#`

`# 예제3`: how

In [131]:
df1 = pd.DataFrame({
    'dept':['통계','수학','과학','IAB'], 
    'count':[20,30,25,50]
})
df2 = pd.DataFrame({
    'dept':['통계','수학','과학','신설학과'], 
    'desc':['통계학과는...','수학과는...','과학학과는...','이 학과는 내년에 신설되 예정이고...']
})
display("df1",df1)
display("df2",df2)

-   공통의열인 `dept`와 서로다른 정보인 `count`, `desc` 들의 이름이
    예쁘게 정리되어 있어 `on`, `left_on`, `right_on` 와 같은 파라메터를
    사용할 필요가 없다.

In [137]:
pd.merge(df1,df2) # IAB, 신설학과는 사라짐

`-` 상황: IAB 학과는 df1에만, 신설학과는 df2에만 존재한다. $\to$ IAB와
신설학과를 처리하는 방식에 따라서 4가지 경우로 나누어진다.

In [145]:
# 경우1: 두 학과 모두 제거 = 두 데이터프레임의 공통만 취함
pd.merge(df1,df2)
pd.merge(df1,df2,how='inner')

In [146]:
# 경우2: IAB만 살림 = 왼쪽 데이터프레임에 포함된 모든 학과는 살림
pd.merge(df1,df2)
pd.merge(df1,df2,how='left')

In [144]:
# 경우3: 신설학과만 살람 = 오른쪽 데이터프레임에 포함된 모든학과를 살람
pd.merge(df1,df2)
pd.merge(df1,df2,how='right')

In [147]:
# 경우4: 두 학과 모두 살림
pd.merge(df1,df2)
pd.merge(df1,df2,how='outer')

# 4. `concat`, `merge` 를 이용한 데이터 합치기

`# 예제` – 전북대학교 통계학과 교과목

주어진 자료가 아래와 같다.

In [153]:
df_course2023 = pd.DataFrame({
    'name':['최규빈']*3+['최혜미']*2+['이영미']+['양성준'],
    'year':[2023]*7,
    'course':['파이썬프로그래밍', '데이터시각화', '기계학습활용','수리통계1', '수리통계2','회귀분석1','통계수학']})
df_course2023

In [154]:
df_course2024 = pd.DataFrame({
    'name':['최규빈','이영미','이영미','양성준','최혜미'],
    'year':[2024]*5,
    'course':['기계학습활용','수리통계1', '수리통계2','회귀분석1','통계수학']})
df_course2024

In [150]:
df_sex = pd.DataFrame({'name':['최규빈','이영미','양성준','최혜미'],
                        'sex':['male','female','male','female']})
df_sex

In [157]:
df_score = pd.DataFrame({
    'name':['최규빈','최규빈','이영미','이영미','양성준','양성준','최혜미','최혜미'],
    'year':[2023,2024]*4,
    'score':[1, 1.2, 5,5,5,5,5,5]})

In [158]:
df_score

주어진 정보를 바탕으로, 4개의 데이터 프레임을 결합하라.

(풀이)

In [162]:
pd.concat([df_course2023,df_course2024])\
.merge(df_score,on=['name','year'])\
.merge(df_sex,on='name')

`#`